In [1]:
import pyarrow as pa
import io
import gzip
from pyarrow import json as pajson
from pyarrow.json import ReadOptions, ParseOptions
import pyarrow.parquet as pq

In [2]:
# use if schema is unknown and you want to write entire table
def get_schema(fn):
    return pajson.read_json(fn).schema

In [3]:
sch = get_schema('test_0.json')

In [4]:
def stream_json(fns, schema, chunk_size=10000000, is_gzip=False):
    writer = pq.ParquetWriter('sample.parquet', schema)
    
    for fn in fns:
        if is_gzip:
            f = gzip.open(fn, 'r')
        else:
            f = open(fn, 'r')
            
        with f:
            while True:
                chunk = f.readlines(chunk_size)
                if not chunk:
                    break
                    
                tbl = pajson.read_json(io.BytesIO(''.join(chunk).encode()))
                
                # insert code to edit tbl for writing schema
                
                assert tbl.schema == schema
                writer.write_table(tbl)
                
            f.close()
            
    writer.close()

In [5]:
fns = ['test_0.json', 'test_1.json', 'test_2.json', 'test_3.json']

Each file is approx 660MB with 13M lines of a repeating ndjson value: {"some":"thing","foo":17,"bar":false,"quux":true}

In [6]:
%time stream_json(fns, sch)

Wall time: 37 s


Resulting parquet file is 880KB (likely due to excellent compression).